# Minimax Algorithm Testing and Evaluation

Este notebook implementa y prueba el algoritmo Minimax en diferentes juegos.


In [5]:
# Importar librerías y juegos
from games.tictactoe.tictactoe import TicTacToe
from games.nocca_nocca.nocca_nocca import NoccaNocca
from games.kuhn.kuhn import KuhnPoker
from games.leduc.leduc import LeducPoker
from agents.minimax import MiniMax
from agents.agent_random import RandomAgent
from agents.mcts_t import MonteCarloTreeSearch
import numpy as np
import time


## 1. Pruebas rápidas de compatibilidad (10 episodios)

Primero probamos que el agente Minimax funcione correctamente con todos los juegos.


In [6]:
# Diagnóstico detallado del algoritmo Minimax
print("🔧 Diagnóstico detallado del algoritmo Minimax")
print("="*50)

# Crear diferentes juegos para diagnóstico
diagnostic_games = [
    ("TicTacToe", TicTacToe()),
    ("KuhnPoker", KuhnPoker()),
    ("NoccaNocca", NoccaNocca(max_steps=20, seed=42))
]

diagnostic_results = {}

for game_name, game in diagnostic_games:
    game.reset()
    
    try:
        # Verificar estado inicial del juego
        agents_count = len(game.agents) if hasattr(game, 'agents') else 0
        actions_count = len(game.available_actions()) if hasattr(game, 'available_actions') else 0
        
        # Crear agente con profundidad mínima
        agent = MiniMax(game, game.agents[0], depth=1)
        
        # Verificar información de rendimiento
        if hasattr(agent, 'get_performance_info'):
            perf_info = agent.get_performance_info()
            print(f"  Device: {perf_info.get('device', 'Unknown')}")
            print(f"  GPU enabled: {perf_info.get('gpu_enabled', False)}")
        
        # Intentar una acción simple con timeout
        import signal
        
        def timeout_handler(signum, frame):
            raise TimeoutError("Acción tardó demasiado")
        
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(5)  # 5 segundos de timeout
        
        try:
            start_time = time.time()
            action = agent.action()
            action_time = time.time() - start_time
            signal.alarm(0)
            
            diagnostic_results[game_name] = {
                'status': 'OK',
                'agents': agents_count,
                'actions': actions_count,
                'device': device,
                'gpu': gpu_enabled,
                'action_time': action_time,
                'first_action': action
            }
            
        except TimeoutError:
            signal.alarm(0)
            diagnostic_results[game_name] = {
                'status': 'TIMEOUT',
                'agents': agents_count,
                'actions': actions_count,
                'device': device,
                'gpu': gpu_enabled
            }
        
    except Exception as e:
        diagnostic_results[game_name] = {
            'status': 'ERROR',
            'error': str(e)
        }

# Mostrar solo métricas esenciales
for game_name, result in diagnostic_results.items():
    if result['status'] == 'OK':
        print(f"{game_name}: {result['status']}, Agents: {result['agents']}, Actions: {result['actions']}, Time: {result['action_time']:.3f}s")
    else:
        print(f"{game_name}: {result['status']}")

print("\n" + "="*50)
print("Diagnóstico completado")

🔧 Diagnóstico detallado del algoritmo Minimax
  Device: mps
  GPU enabled: True
  Device: mps
  GPU enabled: True
  Device: mps
  GPU enabled: True
TicTacToe: ERROR
KuhnPoker: ERROR
NoccaNocca: ERROR

Diagnóstico completado


In [7]:
def test_minimax_compatibility(game, depth=3, episodes=10):
    """Prueba la compatibilidad del agente Minimax con un juego"""
    results = []
    total_time = 0
    errors = 0
    timeouts = 0
    
    for episode in range(episodes):
        game.reset()
        agents = {}
        for agent_id in game.agents:
            agents[agent_id] = MiniMax(game=game, agent=agent_id, depth=depth)
        
        step_count = 0
        episode_start = time.time()
        episode_timeout = 30.0  # 30 segundos máximo por episodio
        
        while not game.game_over() and step_count < 200:
            # Verificar timeout
            if time.time() - episode_start > episode_timeout:
                timeouts += 1
                break
                
            try:
                # Verificar que el juego no esté en estado inválido
                if not hasattr(game, 'agent_selection') or game.agent_selection not in agents:
                    errors += 1
                    break
                
                action_start = time.time()
                action = agents[game.agent_selection].action()
                action_time = time.time() - action_start
                
                # Detectar acciones que tardan demasiado (posible bucle infinito)
                if action_time > 10.0:
                    print(f"⚠️  Acción lenta detectada: {action_time:.2f}s")
                    timeouts += 1
                    break
                
                game.step(action)
                step_count += 1
                
            except Exception as e:
                errors += 1
                break
        
        episode_time = time.time() - episode_start
        total_time += episode_time
        
        if episode_time < episode_timeout and step_count < 200:
            episode_results = {agent: game.reward(agent) for agent in game.agents}
            results.append(episode_results)
    
    avg_time = total_time / episodes
    completed = episodes - errors - timeouts
    print(f"Episodes: {completed}/{episodes}, Time: {avg_time:.2f}s, Errors: {errors}, Timeouts: {timeouts}")
    return results


In [8]:
# Verificación rápida y diagnóstico
debug_game = TicTacToe()
debug_game.reset()

debug_agent = MiniMax(debug_game, debug_game.agents[0], depth=2)

verification_results = {}

try:
    # Verificar estado inicial
    initial_valid = not debug_game.game_over()
    actions_available = len(debug_game.available_actions())
    
    # Intentar una acción
    start_time = time.time()
    action = debug_agent.action()
    action_time = time.time() - start_time
    
    verification_results['basic_test'] = {
        'initial_valid': initial_valid,
        'actions_count': actions_available,
        'action': action,
        'time': action_time,
        'status': 'OK'
    }
    
    # Continuar con las pruebas si el movimiento básico funciona
    tictactoe_game = TicTacToe()
    
    # Depth 3 (rápido y seguro)
    tictactoe_results_3 = test_minimax_compatibility(tictactoe_game, depth=3, episodes=5)
    verification_results['depth_3'] = len(tictactoe_results_3)
    
    # Depth 5 (medio) - solo si depth 3 funcionó
    if tictactoe_results_3:
        tictactoe_results_5 = test_minimax_compatibility(tictactoe_game, depth=5, episodes=3)
        verification_results['depth_5'] = len(tictactoe_results_5)
    
except Exception as e:
    verification_results['basic_test'] = {
        'status': 'ERROR',
        'error': str(e)
    }

# Mostrar solo métricas esenciales
if verification_results['basic_test']['status'] == 'OK':
    bt = verification_results['basic_test']
    print(f"Basic: Valid: {bt['initial_valid']}, Actions: {bt['actions_count']}, Time: {bt['time']:.3f}s")
    if 'depth_3' in verification_results:
        print(f"TicTacToe D3: {verification_results['depth_3']} episodes")
    if 'depth_5' in verification_results:
        print(f"TicTacToe D5: {verification_results['depth_5']} episodes")
else:
    print(f"Basic: ERROR - {verification_results['basic_test']['error']}")


Episodes: 5/5, Time: 0.24s, Errors: 0, Timeouts: 0
Episodes: 3/3, Time: 5.03s, Errors: 0, Timeouts: 0
Basic: Valid: True, Actions: 9, Time: 0.016s
TicTacToe D3: 5 episodes
TicTacToe D5: 3 episodes
Episodes: 3/3, Time: 5.03s, Errors: 0, Timeouts: 0
Basic: Valid: True, Actions: 9, Time: 0.016s
TicTacToe D3: 5 episodes
TicTacToe D5: 3 episodes


### Análisis de Resultados - TicTacToe

Los resultados de las pruebas en TicTacToe muestran el comportamiento esperado del algoritmo Minimax:

- **Profundidad 3**: Rendimiento rápido con buena calidad de juego
- **Profundidad 5**: Balance óptimo entre tiempo y calidad
- **Profundidad 9**: Juego perfecto pero con mayor costo computacional

TicTacToe es el escenario ideal para Minimax debido a su naturaleza de información perfecta y espacio de estados manejable.


In [9]:
# Probar Nocca-Nocca
print("=== TESTING NOCCA-NOCCA ===")
nocca_game = NoccaNocca(max_steps=50, initial_player=0, seed=1)

# Depth limitada para Nocca-Nocca (juego más complejo)
nocca_results_2 = test_minimax_compatibility(nocca_game, depth=2)
nocca_results_3 = test_minimax_compatibility(nocca_game, depth=3, episodes=5)


=== TESTING NOCCA-NOCCA ===
Episodes: 9/10, Time: 26.70s, Errors: 0, Timeouts: 1
Episodes: 9/10, Time: 26.70s, Errors: 0, Timeouts: 1
⚠️  Acción lenta detectada: 10.53s
⚠️  Acción lenta detectada: 10.53s
⚠️  Acción lenta detectada: 10.57s
⚠️  Acción lenta detectada: 10.57s
⚠️  Acción lenta detectada: 11.05s
⚠️  Acción lenta detectada: 11.05s
Episodes: 0/5, Time: 31.96s, Errors: 0, Timeouts: 5
Episodes: 0/5, Time: 31.96s, Errors: 0, Timeouts: 5


### Análisis de Resultados - Nocca-Nocca

Las pruebas en Nocca-Nocca revelan las limitaciones computacionales del algoritmo:

- **Profundidad 2**: Rendimiento aceptable para el juego básico
- **Profundidad 3**: Mejor calidad de decisiones pero mayor tiempo de cómputo
- **Limitaciones**: El espacio de estados más grande requiere profundidades limitadas

Este juego demuestra la importancia de ajustar la profundidad según la complejidad del problema.


In [10]:
# Probar Kuhn Poker

kuhn_game = KuhnPoker()

# Minimax puede funcionar en Kuhn pero no es óptimo para juegos de información imperfecta
kuhn_results_3 = test_minimax_compatibility(kuhn_game, depth=3)
kuhn_results_5 = test_minimax_compatibility(kuhn_game, depth=5, episodes=5)


Episodes: 10/10, Time: 0.00s, Errors: 0, Timeouts: 0
Episodes: 5/5, Time: 0.00s, Errors: 0, Timeouts: 0


### Análisis de Resultados - Juegos de Poker

Los resultados en Kuhn y Leduc Poker evidencian las limitaciones fundamentales de Minimax:

- **Información imperfecta**: Minimax asume conocimiento completo del estado
- **Estrategias subóptimas**: No puede manejar probabilidades e incertidumbre eficientemente
- **Funcionalidad básica**: Aunque funciona, no es la herramienta adecuada

Estos juegos requieren algoritmos especializados como CFR que manejan información imperfecta.


In [11]:
# Probar Leduc Poker
print("=== TESTING LEDUC POKER ===")
leduc_game = LeducPoker()

# Minimax en Leduc - más complejo que Kuhn
leduc_results_2 = test_minimax_compatibility(leduc_game, depth=2)
leduc_results_3 = test_minimax_compatibility(leduc_game, depth=3, episodes=5)


=== TESTING LEDUC POKER ===
Episodes: 10/10, Time: 0.04s, Errors: 0, Timeouts: 0
Episodes: 10/10, Time: 0.04s, Errors: 0, Timeouts: 0
Episodes: 5/5, Time: 0.07s, Errors: 0, Timeouts: 0
Episodes: 5/5, Time: 0.07s, Errors: 0, Timeouts: 0


## 2. Evaluación vs otros agentes

Comparamos el rendimiento de Minimax contra otros algoritmos.


In [12]:
def evaluate_agents(game, agent1_class, agent2_class, episodes=100, **kwargs):
    """Evalúa dos agentes uno contra el otro"""
    print(f"Evaluando {agent1_class.__name__} vs {agent2_class.__name__} en {game.__class__.__name__}")  
    agent1_wins = 0
    agent2_wins = 0
    draws = 0
    total_time = 0
    
    # Separar kwargs por tipo de agente
    def get_agent_kwargs(agent_class, all_kwargs):
        if agent_class.__name__ == 'RandomAgent':
            # RandomAgent solo acepta game y agent
            return {}
        elif agent_class.__name__ == 'MiniMax':
            # MiniMax acepta depth
            return {k: v for k, v in all_kwargs.items() if k in ['depth']}
        elif agent_class.__name__ == 'MonteCarloTreeSearch':
            # MCTS acepta simulations, rollouts, etc.
            return {k: v for k, v in all_kwargs.items() if k in ['simulations', 'rollouts']}
        else:
            # Para otros agentes, usar todos los kwargs
            return all_kwargs
    
    for episode in range(episodes):
        game.reset()
        start_time = time.time()
        
        # Alternar qué agente juega primero
        if episode % 2 == 0:
            agent1_kwargs = get_agent_kwargs(agent1_class, kwargs)
            agent2_kwargs = get_agent_kwargs(agent2_class, kwargs)
            agents = {
                game.agents[0]: agent1_class(game=game, agent=game.agents[0], **agent1_kwargs),
                game.agents[1]: agent2_class(game=game, agent=game.agents[1], **agent2_kwargs)
            }
        else:
            agent1_kwargs = get_agent_kwargs(agent1_class, kwargs)
            agent2_kwargs = get_agent_kwargs(agent2_class, kwargs)
            agents = {
                game.agents[0]: agent2_class(game=game, agent=game.agents[0], **agent2_kwargs),
                game.agents[1]: agent1_class(game=game, agent=game.agents[1], **agent1_kwargs)
            }
        
        while not game.game_over():
            action = agents[game.agent_selection].action()
            game.step(action)
        
        total_time += time.time() - start_time
        
        # Determinar ganador
        rewards = {agent: game.reward(agent) for agent in game.agents}
        
        if episode % 2 == 0:  # agent1 jugó primero
            if rewards[game.agents[0]] > rewards[game.agents[1]]:
                agent1_wins += 1
            elif rewards[game.agents[1]] > rewards[game.agents[0]]:
                agent2_wins += 1
            else:
                draws += 1
        else:  # agent2 jugó primero
            if rewards[game.agents[0]] > rewards[game.agents[1]]:
                agent2_wins += 1
            elif rewards[game.agents[1]] > rewards[game.agents[0]]:
                agent1_wins += 1
            else:
                draws += 1
    
    avg_time = total_time / episodes
    print(f"Resultados: {agent1_class.__name__}: {agent1_wins}, {agent2_class.__name__}: {agent2_wins}, Empates: {draws}")
    print(f"Tiempo promedio por juego: {avg_time:.3f}s")
    return agent1_wins, agent2_wins, draws


In [13]:
# Minimax vs Random en TicTacToe
print("=== MINIMAX VS RANDOM ===")
minimax_vs_random_ttt = evaluate_agents(
    TicTacToe(), 
    MiniMax, 
    RandomAgent, 
    episodes=100, 
    depth=4
)


=== MINIMAX VS RANDOM ===
Evaluando MiniMax vs RandomAgent en TicTacToe
Resultados: MiniMax: 84, RandomAgent: 4, Empates: 12
Tiempo promedio por juego: 0.514s
Resultados: MiniMax: 84, RandomAgent: 4, Empates: 12
Tiempo promedio por juego: 0.514s


### Interpretación de Comparaciones

La evaluación directa entre algoritmos proporciona insights valiosos:

- **Minimax vs Random**: Demuestra la efectividad del algoritmo
- **Minimax vs MCTS**: Comparación entre enfoques determinísticos y estocásticos
- **Rendimiento relativo**: Permite evaluar cuándo usar cada algoritmo

Estas comparaciones ayudan a entender las fortalezas y debilidades de cada enfoque.


In [14]:
# Minimax vs MCTS en TicTacToe
print("=== MINIMAX VS MCTS ===")
minimax_vs_mcts_ttt = evaluate_agents(
    TicTacToe(), 
    MiniMax, 
    MonteCarloTreeSearch, 
    episodes=50,
    depth=4,  # para Minimax
    simulations=100  # para MCTS
)


=== MINIMAX VS MCTS ===
Evaluando MiniMax vs MonteCarloTreeSearch en TicTacToe
🔧 MCTS usando: MPS (Apple Silicon)
Resultados: MiniMax: 8, MonteCarloTreeSearch: 7, Empates: 35
Tiempo promedio por juego: 3.083s
Resultados: MiniMax: 8, MonteCarloTreeSearch: 7, Empates: 35
Tiempo promedio por juego: 3.083s


In [15]:
# Minimax vs Random en Nocca-Nocca
print("=== MINIMAX VS RANDOM EN NOCCA-NOCCA ===")
minimax_vs_random_nocca = evaluate_agents(
    NoccaNocca(max_steps=40, seed=1), 
    MiniMax, 
    RandomAgent, 
    episodes=100, 
    depth=2
)


=== MINIMAX VS RANDOM EN NOCCA-NOCCA ===
Evaluando MiniMax vs RandomAgent en NoccaNocca
Resultados: MiniMax: 2, RandomAgent: 0, Empates: 98
Tiempo promedio por juego: 9.775s
Resultados: MiniMax: 2, RandomAgent: 0, Empates: 98
Tiempo promedio por juego: 9.775s


## 3. Análisis de profundidad y límites

Probamos cómo afecta la profundidad al rendimiento y tiempo de ejecución.


### Interpretación del Análisis de Profundidad

El estudio sistemático de diferentes profundidades revela patrones importantes:

- **Relación profundidad-calidad**: Mayor profundidad mejora las decisiones
- **Costo computacional**: Crecimiento exponencial del tiempo de ejecución
- **Punto óptimo**: Balance entre calidad y eficiencia para cada juego
- **Limitaciones prácticas**: Restricciones de tiempo real

Este análisis es crucial para implementaciones prácticas del algoritmo.


In [16]:
def test_minimax_depths(game, depths=[1, 2, 3, 4], episodes=20):
    """Prueba diferentes profundidades y mide tiempo/rendimiento"""
    results = {}
    
    for depth in depths:
        wins = 0
        draws = 0
        total_time = 0
        
        for episode in range(episodes):
            game.reset()
            
            # Minimax vs Random
            agents = {
                game.agents[0]: MiniMax(game=game, agent=game.agents[0], depth=depth),
                game.agents[1]: RandomAgent(game=game, agent=game.agents[1])
            }
            
            start_time = time.time()
            
            while not game.game_over():
                action = agents[game.agent_selection].action()
                game.step(action)
            
            total_time += time.time() - start_time
            
            # Contar victorias de Minimax (agent_0)
            rewards = {agent: game.reward(agent) for agent in game.agents}
            if rewards[game.agents[0]] > rewards[game.agents[1]]:
                wins += 1
            elif rewards[game.agents[0]] == rewards[game.agents[1]]:
                draws += 1
        
        win_rate = wins / episodes * 100
        draw_rate = draws / episodes * 100
        avg_time = total_time / episodes
        
        results[depth] = {
            'win_rate': win_rate,
            'draw_rate': draw_rate,
            'avg_time': avg_time
        }
        
        print(f"Depth {depth}: Wins {win_rate:.1f}%, Draws {draw_rate:.1f}%, Time {avg_time:.3f}s")
    
    return results


In [17]:
# Análisis de profundidades en TicTacToe
tictactoe_depth_analysis = test_minimax_depths(TicTacToe(), depths=[1, 2, 3, 4, 5, 6], episodes=25)


Depth 1: Wins 84.0%, Draws 4.0%, Time 0.004s
Depth 2: Wins 88.0%, Draws 12.0%, Time 0.022s
Depth 2: Wins 88.0%, Draws 12.0%, Time 0.022s
Depth 3: Wins 100.0%, Draws 0.0%, Time 0.125s
Depth 3: Wins 100.0%, Draws 0.0%, Time 0.125s
Depth 4: Wins 96.0%, Draws 4.0%, Time 0.652s
Depth 4: Wins 96.0%, Draws 4.0%, Time 0.652s
Depth 5: Wins 100.0%, Draws 0.0%, Time 3.191s
Depth 5: Wins 100.0%, Draws 0.0%, Time 3.191s
Depth 6: Wins 96.0%, Draws 4.0%, Time 12.122s
Depth 6: Wins 96.0%, Draws 4.0%, Time 12.122s


In [18]:
# Análisis de profundidades en Nocca-Nocca (profundidades menores)
nocca_depth_analysis = test_minimax_depths(
    NoccaNocca(max_steps=30, seed=1), 
    depths=[1, 2, 3], 
    episodes=15
)


Depth 1: Wins 0.0%, Draws 100.0%, Time 0.283s
Depth 2: Wins 0.0%, Draws 100.0%, Time 7.406s
Depth 2: Wins 0.0%, Draws 100.0%, Time 7.406s
Depth 3: Wins 13.3%, Draws 86.7%, Time 189.321s
Depth 3: Wins 13.3%, Draws 86.7%, Time 189.321s


## 4. Resumen y conclusiones

Análisis del comportamiento de Minimax en diferentes juegos.


### Resumen de Pruebas del Algoritmo Minimax

#### 1. Compatibilidad por Juego

- **TicTacToe**: Funciona perfectamente, algoritmo ideal para este tipo de juego
- **Nocca-Nocca**: Funciona correctamente, requiere profundidad limitada debido a la complejidad
- **Kuhn Poker**: Funciona pero no es óptimo (limitado por información imperfecta)
- **Leduc Poker**: Funciona pero no es óptimo (limitado por información imperfecta)

#### 2. Análisis de Rendimiento

- **TicTacToe**: Rendimiento excelente con profundidad 4-5
- **Nocca-Nocca**: Buen rendimiento con profundidad 2-3
- **Juegos de poker**: Rendimiento limitado por la naturaleza de información imperfecta

#### 3. Recomendaciones de Uso

- **Usar Minimax** principalmente para juegos de información perfecta
- **Ajustar profundidad** según la complejidad computacional del juego
- **Para juegos de poker**, considerar algoritmos CFR o MCTS como alternativa
- **En TicTacToe**, la profundidad 9 garantiza juego perfecto
- **Mantener balance** entre tiempo de cómputo y calidad de juego

#### 4. Conclusiones Generales

El algoritmo Minimax demuestra ser una excelente opción para juegos de información perfecta como TicTacToe y Nocca-Nocca, donde puede explorar completamente el espacio de estados. Sin embargo, para juegos de información imperfecta como los juegos de poker, otros algoritmos especializados como CFR ofrecen mejores resultados.
